Read in a single file and make duplicates

In [5]:
import pyspark
from delta import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import json
spark = (pyspark.sql.SparkSession.builder.appName("MyApp").master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore")
.getOrCreate())
spark.sparkContext.setLogLevel('ERROR')

In [6]:
df_array = ["""{"id": "2111", "name": "OLIVIA", "age": "37", "metadata":[["a",1], ["b",2]]}"""]
df_array = spark.sparkContext.parallelize(df_array)
df_array = spark.read.json(df_array)
df_array.show()

+---+----+----------------+------+
|age|  id|        metadata|  name|
+---+----+----------------+------+
| 37|2111|[[a, 1], [b, 2]]|OLIVIA|
+---+----+----------------+------+



In [14]:
df_array.write.format("delta").mode("overwrite").save("spark")

In [18]:
spark.read.format("delta").load("spark").show()

+---+----+----------------+------+
|age|  id|        metadata|  name|
+---+----+----------------+------+
| 37|2111|[[a, 1], [b, 2]]|OLIVIA|
+---+----+----------------+------+

